In [172]:
import os
import glob
import re
import pandas as pd
import numpy as np
import PyPDF2

path = os.getcwd()
pdf_files = glob.glob(path + "/Protocolos/*.pdf")
print(len(pdf_files))


197


In [173]:
def convert_pdf_to_txt(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfReader(pdf_file)
    number_of_pages = len(read_pdf.pages)
    page_content = ""
    for page_number in range(number_of_pages):
        page = read_pdf.pages[page_number]
        page_content += page.extract_text()
    pdf_file.close()
    return page_content
text = []
index=1
print("Proceso 1")
for i in pdf_files:
    print(f"{index}/{len(pdf_files)}", end='\r')
    text.append(convert_pdf_to_txt(i))
    index+=1

Proceso 1


In [324]:
def get_title(text):
    text = text.split('trabajo terminal')
    text = text[0]
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    #eliminar cualquier texto antes de la primera letra
    text = re.sub(r'^[^a-zA-Z]+', '', text)
    #eliminar todos los espacios antes de la primera letra
    text = re.sub(r'^\s+', '', text)
    return text

def get_ttnum(text):
    text = text.split('trabajo terminal')
    text = text[1]
    # si alum aparece en ek texto
    if 'alumn' in text:
        text = text.split('alumn')
        text = text[0]
    if 'integrantes' in text:
        text = text.split('integrantes')
        text = text[0]
    text = re.sub(' +', '', text)
    # si aparece "no." en text remover
    if 'no.' in text:
        text = text.replace('no.', '')
    elif 'no:' in text:
        text = text.replace('no:', '')
    elif 'no' in text:
        text = text.replace('no', '')
    # remover los '_' de text
    text = text.replace('_', '')
    # remover dos puntos de text
    text = text.replace(':', '')
    if not re.match(r'^\d{4}-[a-zA-Z]\d{3}$', text):
        text = '-'
    return text.upper()

def get_director(text):
    text = text.split('director')
    text = text[1]
    # si e-mail aparece en el texto 
    if 'e-mail' in text:
        text = text.split('e-mail')
        text = text[0]
    elif 'resumen' in text:
        text = text.split('resumen')
        text = text[0]
    # eliminar texto antes de :
    text = text.split(':')
    text = text[1]
    #eliminar :
    text = text.replace(':', '')
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    # remover '*' de text
    text = text.replace('*', '')
    # eliminar el texto despues de la palbra 'turno'
    if 'turno' in text:
        text = text.split('turno')
        text = text[0]
    #eliminar todos los correos del texto
    text = re.sub(r'[\w\.-]+@[\w\.-]+', '', text)
    # split por espacios y los que tengan arrorba ,remplazar todo el texto por @
    text = text.split(' ')
    text = [i.replace(i, '@') if '@' in i else i for i in text]
    text = ' '.join(text)
    # remover texto despues de @
    text = re.sub(r'@.*', '', text)
    #remover caracteres especiales en el texto, excepto ','
    text = re.sub(r'[^\w\s,]', '', text)
    #remover la palabra email del texto
    text = re.sub(r'email', '', text)
    #remover todos los numeros del texto
    text = re.sub(r'\d+', '', text)
    #split text por espacios y eliminar las palabras que son menores a 4 caracteres
    text = text.split(' ')
    text = [i for i in text if len(i) > 3]
    text = ' '.join(text)
    #Split por , 
    text = text.split(',')
    #eliminar espacios en blanco al inicio y al final de cada elemento de la lista
    text = [i.strip() for i in text]
    return text

def get_abstract(text):
    text = text.split('resumen')
    text = text[1]
    text = text.split('palabras clave')
    text = text[0]
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    #eliminar cualquier texto antes de la primera letra
    text = re.sub(r'^[^a-zA-Z]+', '', text)
    #eliminar todos los espacios antes de la primera letra y despues de la ultima letra o punto
    text = re.sub(r'\s+$', '', text)
    return text

def get_keywords(text):
    text = text.split('palabras clave')
    text = text[1]
    text = text.split('.')
    text = text[0]
    text = re.sub(' +', ' ', text)
    #eliminar cualquier texto antes de la primera letra
    text = re.sub(r'^[^a-zA-Z]+', '', text)
    #eliminar todos los espacios antes de la primera letra
    text = re.sub(r'^\s+', '', text)
    #Split por , 
    text = text.split(',')
    # si la celda de keywords es mayor a 20 caracteres eliminarla
    text = [i for i in text if len(i) < 20]
    #eliminar espacios en blanco al inicio y al final de cada elemento de la lista
    text = [i.strip() for i in text]
    return text

def get_areas(text):
    # si existe areas de interes en el texto 
    if 'áreas de interés' in text:
        text = text.split('áreas de interés')
        text = text[1]
        text = text.split('.')
        text = text[0]
        text = text.replace('\n', '')
        text = re.sub(' +', ' ', text)
        #eliminar cualquier texto antes de la primera letra
        text = re.sub(r'^[^a-zA-Z]+', '', text)
        #eliminar todos los espacios antes de la primera letra
        text = re.sub(r'^\s+', '', text)
        #Split por , 
        text = text.split(',')
        # remover 'son:' de text
        text = [i.replace('son:', '') for i in text]
        #eliminar espacios en blanco al inicio y al final de cada elemento de la lista
        text = [i.strip() for i in text]
        # eliminar toda la celda que contenga tel, ext, correo, e-mail en areas de interes
        text = [i for i in text if 'tel' not in i]
        text = [i for i in text if 'ext' not in i]
        text = [i for i in text if 'correo' not in i]
        text = [i for i in text if 'e-mail' not in i]
        text = [i for i in text if '-' not in i]
        text = [i for i in text if 'email' not in i]
        #si la area de interes es menor a 50 caracteres eliminarla
        text = [i for i in text if len(i) < 50]
        # remover las celdas vacias de areas de interes
        text = [i for i in text if i != '']

        if len(text) == 0:
            text = '-'
        return text
    else:
        return '-'

def get_objetivo(text):
    text = text.split('objetivo')
    text = text[1]
    text = text.split('.')
    text = text[0]
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

def get_objetivos_especificos(text):
    # si existe "objetivos especificos" o "especificos" en el texto
    if 'objetivos específicos' in text or 'específicos' in text:
        if 'objetivos específicos' in text:
            text = text.split('objetivos específicos')
            text = text[1]
            text = text.split('.')
            text = text[0]
            text = text.replace('\n', '')
            text = re.sub(' +', ' ', text)
            return text
        else:
            text = text.split('específicos')
            text = text[1]
            text = text.split('.')
            text = text[0]
            text = text.replace('\n', '')
            text = re.sub(' +', ' ', text)
            return text

In [325]:
#pasar todos los text a minusculas 
text = [x.lower() for x in text]
# quitar todos los "\n" de text
text = [x.replace('\n', '') for x in text]
# quitar dobles espacios
text = [re.sub(' +', ' ', x) for x in text]

In [326]:
titulos = []
num_tt = []
abstracts = []
keywords = []
directores = []
areas_interes = []
objetivos = []
objetivos_especificos=[]
index=1

for j in text:
    print(f"{index}/{len(pdf_files)}", end='\r')
    titulos.append(get_title(j))
    num_tt.append(get_ttnum(j))
    keywords.append(get_keywords(j))
    directores.append(get_director(j))
    areas_interes.append(get_areas(j))
    abstracts.append(get_abstract(j))
    
    objetivos.append(get_objetivo(j))
    objetivos_especificos.append(get_objetivos_especificos(j))
    index+=1


In [330]:
# guardar en un archivo de excel con las columnas titulo, N_TT, abstract, keywords, directores, objetivos
df = pd.DataFrame({'titulo': titulos , 'N_TT': num_tt,'Directores': directores, 'keywords':keywords, 'areas_interes':areas_interes, 'abstract':abstracts})
df.fillna('-', inplace=True)
# save the dataframe in a excel with the name "extraccion"
df.to_excel('extraccion.xlsx', index=False)

df.head()

,titulo,N_TT,Directores,keywords,areas_interes,abstract
0,prototipo de aplicación móvil para temas de f ...,2019-B037,[ruíz ledesma elena fabiola],"[realidad aumentada, m -learning, física mecán...",-,la propuesta consiste en el desarrollo de una ...
1,framework para la implementación de interfaz g...,2019-A053,"[pescador rojas miriam, coronilla contreras uk...","[framework, c++]",-,el propósito de este trabajo terminal es progr...
2,monedero electronico para cafeterias escolares...,-,"[amadeo josé argüelles cruz, alfredo rangel gu...","[sistema embebido, aplicación móvil, tecnológi...","[sistemas embebidos, logica reconfigurable]",en el presente trabajo se propone implementar ...
3,herramienta para la creación de un entorno per...,-,[carreto arellano chadwick bustos farías eduardo],"[pedagogía, desarrollo web]","[cómputo móvil, redes]",con este trabajo terminal (tt) se pretende cre...
4,software de evaluación de datos de espectromet...,2016-B015,[tonáhtiu arturo ramírez romero miguel hesiqui...,[análisis de señales],"[bases de datos, desarrollo de sistemas web y ...",este trabajo tiene la finalidad de crear un so...


In [318]:
# procesar df con spacy 
import spacy
from spacy.lang.es.examples import sentences 
# !python -m spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")

titulos_lema = []

# para cada titulo de los titulos procesar con nlp
for i in df['titulo']:
    doc = nlp(i)
    # para cada token en el doc 
    titulo_aux = ''
    for token in doc:
        # lematizar el token
        titulo_aux += token.lemma_ + ' '
    titulos_lema.append(titulo_aux)
titulos_lema   
            

['prototipo de aplicación móvil para tema de f ísico usar realidad aumentado ',
 'framework para el implementación de interfaz gráfico de usuario : guinox ',
 'monedero electronico para cafeteria escolar , por medio de uno sistema embebido ',
 'herramienta para el creación de uno entorno personal de aprendizaje ',
 'software de evaluación de dato de espectrometría infrar rojo ',
 'prototipo de “ sistema para el gestión y administración de el proceso de inscripción en cocendi ” ',
 'herramienta para el vinculación laboral ',
 'nodo sensor inalámbrico autoalimentado por cosecha de energía aplicado a domótico . ',
 'prototipo de control de seguimiento para el departamento de coordinación de programa de el unidad politécnico de educación virtual . ',
 'prototipo de h erramienta de apoyo para el detección y tratamiento del estrabismo mediante realidad virtual : eyetec dts ',
 'prototipo para el geolocalización de vehículo . ',
 'prototipo de bajo costo para aplicación iot ',
 'prototipo de 